In [1]:
from ner import NER

C:\Users\linhpn.VISC\AppData\Local\Continuum\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
parameters = {'pretrained_model_folder':'../model',
                      'dataset_text_folder':'../../../ML_EntityData/data/en',
                      'character_embedding_dimension':25,
                      'character_lstm_hidden_state_dimension':25,
                      'check_for_digits_replaced_with_zeros':True,
                      'check_for_lowercase':True,
                      'debug':False,
                      'dropout_rate':0.5,
                      'experiment_name':'test',
                      'freeze_token_embeddings':False,
                      'gradient_clipping_value':5.0,
                      'learning_rate':0.005,
                      'load_only_pretrained_token_embeddings':False,
                      'load_all_pretrained_token_embeddings':False,
#                       'main_evaluation_mode':'conll',
                      'maximum_number_of_epochs':500,
                      'number_of_cpu_threads':8,
                      'number_of_gpus':0,
                      'optimizer':'sgd',
                      'output_folder':'../../../ML_EntityData/output',
                      'patience':10,
#                       'plot_format':'pdf',
                      'reload_character_embeddings':True,
                      'reload_character_lstm':True,
                      'reload_crf':True,
                      'reload_feedforward':True,
                      'reload_token_embeddings':True,
                      'reload_token_lstm':True,
                      'remap_unknown_tokens_to_unk':True,
#                       'spacylanguage':'en',
                      'tagging_format':'bioes',
                      'token_embedding_dimension':100,
                      'token_lstm_hidden_state_dimension':100,
                      'token_pretrained_embedding_filepath':'../../../ML_EntityData/embedding/en/glove.6B.100d.txt',
#                       'tokenizer':'spacy',
                      'train_model':True,
                      'use_character_lstm':True,
#                       'use_crf':True,
                      'use_pretrained_model':False,
                      'verbose':False}

In [3]:
ner = NER(parameters)

Checking the validity of BRAT-formatted train set... Done.
Checking compatibility between CONLL and BRAT for train_compatible_with_brat set ... Done.
Checking validity of CONLL BIOES format... Done.
Checking the validity of BRAT-formatted valid set... Done.
Checking compatibility between CONLL and BRAT for valid_compatible_with_brat set ... Done.
Checking validity of CONLL BIOES format... Done.
Checking the validity of BRAT-formatted test set... Done.
Checking compatibility between CONLL and BRAT for test_compatible_with_brat set ... Done.
Checking validity of CONLL BIOES format... Done.
Load dataset... done (100.86 seconds)
embedded_characters: Tensor("character_embedding/embedded_characters:0", shape=(?, ?, 25), dtype=float32)
embedded_tokens: Tensor("token_embedding/embedding_lookup:0", shape=(?, 100), dtype=float32)
token_lstm_input: Tensor("concatenate_token_and_character_vectors/token_lstm_input:0", shape=(?, 163), dtype=float32)
token_lstm_input_drop: Tensor("dropout/token_lstm_

C:\Users\linhpn.VISC\AppData\Local\Continuum\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Load token embeddings... done (0.16 seconds)
number_of_token_original_case_found: 14618
number_of_token_lowercase_found: 11723
number_of_token_digits_replaced_with_zeros_found: 119
number_of_token_lowercase_and_digits_replaced_with_zeros_found: 16
number_of_loaded_word_vectors: 26476
dataset.vocabulary_size: 28984


In [20]:
ner.restore_model_trained(parameters,model_pathfile=('../model/en/model_00000.ckpt'),
                                        dataset_pathfile=('../model/en/dataset.pickle'),
                                        embedding_filepath= parameters['token_pretrained_embedding_filepath'],
                                        character_dimension = parameters['character_embedding_dimension'],
                                        token_dimension=parameters['token_embedding_dimension'])

INFO:tensorflow:Restoring parameters from ../model/en/model_00000.ckpt


In [22]:
ner.restore_model_trained(parameters,model_pathfile=('../../../ML_EntityData/output/en_2018-02-28_13-55-28-670626/model/model.ckpt'),
                                        dataset_pathfile=('../../../ML_EntityData/output/en_2018-02-28_13-55-28-670626/model/dataset.pickle'),
                                        embedding_filepath= parameters['token_pretrained_embedding_filepath'],
                                        character_dimension = parameters['character_embedding_dimension'],
                                        token_dimension=parameters['token_embedding_dimension'])

INFO:tensorflow:Restoring parameters from ../../../ML_EntityData/output/en_2018-02-28_13-55-28-670626/model/model.ckpt


In [37]:
ner.train(parameters,0,'../../../ML_EntityData/output/en/model')


Starting epoch 0
Training completed in 0.00 seconds
Evaluate model on the train set
             precision    recall  f1-score   support

      B-LOC     0.9990    0.9990    0.9990      1041
     B-MISC     0.9953    0.9942    0.9948       858
      B-ORG     0.9992    0.9976    0.9984      2485
      B-PER     0.9995    0.9995    0.9995      4284
      E-LOC     0.9990    0.9990    0.9990      1041
     E-MISC     0.9988    0.9977    0.9983       858
      E-ORG     0.9992    0.9976    0.9984      2485
      E-PER     0.9995    0.9995    0.9995      4284
      I-LOC     1.0000    1.0000    1.0000       116
     I-MISC     0.9900    1.0000    0.9950       297
      I-ORG     1.0000    0.9975    0.9988      1219
      I-PER     1.0000    1.0000    1.0000       244
          O     0.0000    0.0000    0.0000        21
      S-LOC     0.9980    0.9964    0.9972      6099
     S-MISC     0.9961    0.9946    0.9953      2580
      S-ORG     0.9971    0.9964    0.9967      3836
      S-PER  

In [46]:
ner.predict('Ngoc Pham  Linh',parameters)

Formatting deploy set from BRAT to CONLL... Done.
Converting CONLL from BIO to BIOES format... Done.
Predict labels for the deploy set
Formatting 000_deploy set from CONLL to BRAT... Done.

text:
Ngoc Pham  Linh

entity: {'id': 'T1', 'type': 'PER', 'start': 5, 'end': 15, 'text': 'Pham Linh'}





[{'end': 15, 'id': 'T1', 'start': 5, 'text': 'Pham Linh', 'type': 'PER'}]

In [28]:
ner.predict("Hồ Chí Minh Vietnamese:  (About this sound listen), Saigon:  (About this sound listen); Chữ nôm: ; 19 May 1890 – 2 September 1969), born Nguyễn Sinh Cung, also known as Nguyễn Tất Thành and Nguyễn Ái Quốc, was a Vietnamese Communist revolutionary leader who was Chairman and First Secretary of the Workers' Party of Vietnam. Hồ was also Prime Minister (1945–55) and President (1945–69) of the Democratic Republic of Vietnam (North Vietnam). He was a key figure in the foundation of the Democratic Republic of Vietnam in 1945, as well as the People's Army of Vietnam (PAVN) and the Việt Cộng (NLF or VC) during the Vietnam War.Hồ Chí Minh led the Việt Minh independence movement from 1941 onward, establishing the Communist-ruled Democratic Republic of Vietnam in 1945 and defeating the French Union in 1954 at the battle of Điện Biên Phủ. He officially stepped down from power in 1965 due to health problems. After the war, Saigon, the former capital of the Republic of Vietnam, was renamed Hồ Chí Minh City.Any description of Ho's life before he came to power in Vietnam is necessarily fraught with ambiguity. He is known to have used at least 50 and perhaps as many as 200 pseudonyms. (Duiker says at least 75.) His place of birth and date of birth are products of academic consensus since neither is known with certainty. \"Official biographies and there are at least four, vary on names, dates, places and other hard facts. Unofficial biographies vary even more widely.",parameters)

Formatting deploy set from BRAT to CONLL... Done.
Converting CONLL from BIO to BIOES format... Done.
Predict labels for the deploy set
Formatting 000_deploy set from CONLL to BRAT... Done.

text:
Hồ Chí Minh Vietnamese:  (About this sound listen), Saigon:  (About this sound listen); Chữ nôm: ; 19 May 1890 – 2 September 1969), born Nguyễn Sinh Cung, also known as Nguyễn Tất Thành and Nguyễn Ái Quốc, was a Vietnamese Communist revolutionary leader who was Chairman and First Secretary of the Workers' Party of Vietnam. Hồ was also Prime Minister (1945–55) and President (1945–69) of the Democratic Republic of Vietnam (North Vietnam). He was a key figure in the foundation of the Democratic Republic of Vietnam in 1945, as well as the People's Army of Vietnam (PAVN) and the Việt Cộng (NLF or VC) during the Vietnam War.Hồ Chí Minh led the Việt Minh independence movement from 1941 onward, establishing the Communist-ruled Democratic Republic of Vietnam in 1945 and defeating the French Union in 19

[{'end': 22,
  'id': 'T1',
  'start': 0,
  'text': 'Hồ Chí Minh Vietnamese',
  'type': 'MISC'},
 {'end': 153,
  'id': 'T2',
  'start': 137,
  'text': 'Nguyễn Sinh Cung',
  'type': 'PER'},
 {'end': 185, 'id': 'T3', 'start': 176, 'text': 'Tất Thành', 'type': 'PER'},
 {'end': 204, 'id': 'T4', 'start': 197, 'text': 'Ái Quốc', 'type': 'PER'},
 {'end': 232,
  'id': 'T5',
  'start': 212,
  'text': 'Vietnamese Communist',
  'type': 'MISC'},
 {'end': 323,
  'id': 'T6',
  'start': 298,
  'text': "Workers' Party of Vietnam",
  'type': 'ORG'},
 {'end': 423,
  'id': 'T7',
  'start': 393,
  'text': 'Democratic Republic of Vietnam',
  'type': 'ORG'},
 {'end': 438,
  'id': 'T8',
  'start': 425,
  'text': 'North Vietnam',
  'type': 'LOC'},
 {'end': 516,
  'id': 'T9',
  'start': 486,
  'text': 'Democratic Republic of Vietnam',
  'type': 'ORG'},
 {'end': 549, 'id': 'T10', 'start': 541, 'text': "People's", 'type': 'MISC'},
 {'end': 565,
  'id': 'T11',
  'start': 550,
  'text': 'Army of Vietnam',
  'type':